#**TASK 1**

In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.filterwarnings("ignore")


In [71]:
df = pd.read_csv('Amazon Scraping - Sheet1.csv')
df.head()

,Unnamed: 0,id,Asin,country
0,0,1,1015,de
1,1,2424796,1015,fr
2,2,2,000004458X,de
3,3,2424797,000004458X,fr
4,4,3,1002198,de


In [72]:
asin=list(df['Asin'])
country=list(df['country'])

In [73]:
!pip install translators --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
from bs4 import BeautifulSoup
import requests
from urllib.error import HTTPError
import translators as ts
import time


# Function to extract Product Title
def get_title(soup):
	
	try:
		# Outer Tag Object
		title = soup.find("span", attrs={"id":'productTitle'})

		# Inner NavigableString Object
		title_value = title.string

		# Title as a string value
		title_string = title_value.strip()

		# # Printing types of values for efficient understanding
		# print(type(title))
		# print(type(title_value))
		# print(type(title_string))
		# print()

	except AttributeError:
		title_string = ""	

	return title_string

# Function to extract Product Price
def get_price(soup):

	try:
		price = str(soup.find("span", attrs={'class': 'a-price-whole'}))+"."+str(soup.find("span", attrs={'class': 'a-price-fraction'})) #soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()
	except AttributeError:
		  price = "NA"	

	return price

# Function to extract Product Rating
def get_rating(soup):

	try:
		rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
		
	except AttributeError:
		
		try:
			rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
		except:
			rating = "NA"	

	return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
	try:
		review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
		
	except AttributeError:
		review_count = "NA"	

	return review_count

def get_image_url(soup):
	images = []
	for i in soup.findAll('img'):
		if((i.get('src'))!=None):
			images.append(i.get('src'))
	''' 
	try:
		image = soup.findAll('img', attrs={'alt':soup.get_title}).string.strip()
		img = image.get('src')
	except AttributeError:
		img = "NA"
	'''	
	return images 

# Function to extract Availability Status
def get_availability(soup):
	try:
		available = soup.find("div", attrs={'id':'availability'})
		available = available.find("span").string.strip()

	except AttributeError:
		available = "NA"	

	return available	


k = 0
start = time.time()
# Headers for request
products = list()
for i in range(0,1000):
  HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
  URL= "https://www.amazon."+country[i]+"/dp/"+asin[i]
  # HTTP Request
  try:
    k=k+1
    if(k>0 and k%100==0):
      end = time.time()
      print("Time for 100 URLs = ",end-start,"\n")
      start = time.time()
    webpage = requests.get(URL, headers=HEADERS)
  except HTTPError as http_error:
    if (http_error=="HTTP Error 404: Not Found"):
      print(URL+" not available.")
      continue
  else:
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "lxml")
    # Function calls to display all necessary product information
    if(get_title(soup)==""):
      continue
    print("Product Title =", ts.google(get_title(soup)))
    print("Product Image URL =", get_image_url(soup))
    p = str(get_price(soup))
    #print(p)
    s = ""
    for i in range(len(p)):
      if(p[i].isdigit()):
        s=s+p[i]
      if(p[i]==',' or p[i]=='.'):
        s=s+'.'
    print("Product Price =", s)
    print("Product Rating =", ts.google(get_rating(soup)))
    print("Number of Product Reviews =", ts.google(get_review_count(soup)))
    print("Availability =", ts.google(get_availability(soup)))

    data = {'Title': ts.google(get_title(soup)),'Image URLS':get_image_url(soup),'Price':s,'Rating':ts.google(get_rating(soup)),'Number of Reviews':ts.google(get_review_count(soup)),'Availability':ts.google(get_availability(soup))}
    products.append(data)


    print()
    print()




Time for 100 URLs =  28.093990325927734 

Product Title = ENURE 1 & 2 orgue
Product Image URL = ['//fls-eu.amazon.fr/1/batch/1/OP/A13V1IB3VIYZZH:257-0641086-7808731:47DTT5QCJP8PZXV1C2XR$uedata=s:%2Frd%2Fuedata%3Fstaticb%26id%3D47DTT5QCJP8PZXV1C2XR:0', 'https://images-eu.ssl-images-amazon.com/images/G/08/gno/sprites/nav-sprite-global-1x-hm-dsk-reorg._CB405937816_.png', 'https://images-eu.ssl-images-amazon.com/images/G/08/digital/video/gateway/placement/launch/RolandGarros/RG_2022_SWM_POST_XSite_400x39_PV_fr-FR_copy._CB636916982_.jpg', 'https://images-eu.ssl-images-amazon.com/images/G/08/B2B/Merch/GW/ILM/Oct18/ILM_650x45._CB484084574_.jpg', 'https://images-na.ssl-images-amazon.com/images/G/08/x-locale/checkout/truespc/secured-ssl._CB485936936_.png', 'https://images-na.ssl-images-amazon.com/images/G/08/kindle/app/kindle-app-logo._CB666561422_.png', 'https://images-na.ssl-images-amazon.com/images/G/08/kindle/app/QR-store-link-kindle-app._CB626292348_.png', 'https://images-na.ssl-images-ama

In [79]:
import json

final = json.dumps(products, indent=6)
jsonFile = open("data.json", "w")
jsonFile.write(final)
jsonFile.close()
print(final)

[
      {
            "Title": "Old Spice Shaving Cream - 70 g (original) - Pack of 2",
            "Image URLS": [
                  "//fls-eu.amazon.de/1/batch/1/OP/A1PA6795UKMFR9:261-1246257-4049825:AGG66SPYQ1RA8CV65BNF$uedata=s:%2Frd%2Fuedata%3Fstaticb%26id%3DAGG66SPYQ1RA8CV65BNF:0",
                  "https://images-eu.ssl-images-amazon.com/images/G/03/gno/sprites/nav-sprite-global-1x-hm-dsk-reorg._CB405937709_.png",
                  "https://images-eu.ssl-images-amazon.com/images/G/03/apparel/subnav/Arrow_grey_21x13._CB485933084_.png",
                  "https://images-eu.ssl-images-amazon.com/images/G/03/apparel/subnav/Arrow_grey_21x13._CB485933084_.png",
                  "https://images-eu.ssl-images-amazon.com/images/G/03/apparel/subnav/Arrow_grey_21x13._CB485933084_.png",
                  "https://images-eu.ssl-images-amazon.com/images/G/03/apparel/subnav/Arrow_grey_21x13._CB485933084_.png",
                  "https://images-eu.ssl-images-amazon.com/images/G/03/apparel/sub

#**TASK 2**

In [86]:
from bs4 import BeautifulSoup
import requests
from urllib.error import HTTPError

import warnings
warnings.filterwarnings("ignore")

#!pip install amazoncaptcha
from amazoncaptcha import AmazonCaptcha


def get_image_url(soup):
	images = []
	for i in soup.findAll('img'):
		if((i.get('src'))!=None):
			images.append(i.get('src'))
	return images


HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
URL = "https://www.amazon.com/errors/validateCaptcha"
webpage = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "lxml")
img = get_image_url(soup)
#print(img[0])



captcha = AmazonCaptcha.fromlink(img[0])
solution = captcha.solve()
print("Captcha solution: ",solution)
print("Captcha image URL: ",img[0]) 


cap_id = 'captchacharacters'
cap = solution


def submit_form():
    """Submit a form"""
    payload = {cap_id : cap}

    resp = requests.get(URL)
    print("Response to GET request: ",resp.content)

    resp = requests.post(URL, payload)
    print("Headers from a POST request response: ",resp.headers)

submit_form()

Captcha solution:  BRPJPE
Captcha image URL:  https://images-na.ssl-images-amazon.com/captcha/xsqyeruq/Captcha_vwkmeidftd.jpg
Response to GET request:  b'<!DOCTYPE html>\n<!--[if lt IE 7]> <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8 a-lt-ie7"> <![endif]-->\n<!--[if IE 7]>    <html lang="en-us" class="a-no-js a-lt-ie9 a-lt-ie8"> <![endif]-->\n<!--[if IE 8]>    <html lang="en-us" class="a-no-js a-lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html class="a-no-js" lang="en-us"><!--<![endif]--><head>\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<title dir="ltr">Amazon.com</title>\n<meta name="viewport" content="width=device-width">\n<link rel="stylesheet" href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css">\n<script>\n\nif (true === true) {\n    var ue_t0 = (+ new Date()),\n        u